# Noisy simulations with Qiskit

Qiskitを用いてノイズのあるシミュレーションを行う方法を紹介する。
これも、v1で変更があったので理解を深めるために書いてみた。

Qiskitでは、ノイズモデルや実機のcalibration情報のタイムスタンプを用いて、ノイズのあるシミュレーションを行うことができる。
また、具体的な実機に対応したbackendを指定することで、その実機qubitのレイアウトに応じた回路作成や、
native gateへのtranspileを行い、より実際の実機実行に向けたシミュレーションを行うことができる。

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMumbaiV2
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter

In [7]:
def G(circ, i, j, theta):
    theta_2 = theta / 2 
    circ.cx(i,j)
    circ.ry(theta_2, i)
    circ.cx(j,i)
    circ.ry(-theta_2, i)
    circ.cx(j,i)
    circ.cx(i,j)  

def G_gate(theta):
    circ = QuantumCircuit(2)
    G(circ, 0, 1, theta)
    circ.name = "G"    
    return circ.to_gate()

In [8]:
obs = [ "II", "ZI", "IZ", "ZZ"]
coeffs = [ -1.0, 1.0, 1.0, -1.0]
hamiltonian_op = SparsePauliOp(obs, coeffs)
hamiltonian_op

SparsePauliOp(['II', 'ZI', 'IZ', 'ZZ'],
              coeffs=[-1.+0.j,  1.+0.j,  1.+0.j, -1.+0.j])

In [9]:
delta_t = 0.01
n_qubit = 5
U = PauliEvolutionGate(hamiltonian_op, delta_t, synthesis=SuzukiTrotter(order=1,reps=5))
print("U.data", U)

def ansatz(n_qubit):
    ansatz = QuantumCircuit(n_qubit)
    for i in range(2):
        ansatz.x(i)
    ansatz.append(G_gate(np.pi/2), [1,2])
    ansatz.name = "Ansatz"
    return ansatz

def make_cU(Uprep, U, Ntar):
    circuit_cU = QuantumCircuit(Ntar)
    circuit_cU.append(Uprep, range(Ntar))
    circuit_cU.append(U, [1, 2])
    circuit_cU = circuit_cU.decompose(reps=5)
    return circuit_cU.to_gate().control(1)

Uprep = ansatz(n_qubit) 
gate_cU = make_cU(Uprep, U, n_qubit)

qc = QuantumCircuit(1+n_qubit, 1)
qc.h(0)
qc.append(gate_cU, [0] + list(range(1,n_qubit+1)))
qc.x(0)
qc.append(gate_cU, [0] + list(range(1,n_qubit+1)))
qc.h(0)
qc.draw('mpl')
qc = qc.decompose()

U.data Instruction(name='PauliEvolution', num_qubits=2, num_clbits=0, params=[0.01])


Qiskitには、[Fake Provider](https://docs.quantum.ibm.com/api/qiskit/providers_fake_provider)モジュールがあり、様々な実機のcalibration情報を提供している。
これにより、レイアウトに応じた回路作成やtranspileを試したり、ノイズのあるシミュレーションを行うことができる。

`FakeMumbaiV2`を使ってみよう。Mumbaiは27量子ビットのマシンである。
この他にも様々な都市名がついたマシンがあり、量子ビット・ゲートエラー率のほか、processor(≒ネイティブゲート)なども異なるので、興味があれば試してみるとよいだろう。

In [ ]:
backend = FakeMumbaiV2()
tqc = transpile(qc, backend=backend)